In [69]:
import pandas as pd

In [70]:
from sklearn.feature_extraction.text import CountVectorizer
import ast

In [71]:
df = pd.read_csv('RAW_recipes_cleaned.csv')

Tokenize name

Convert stringified ingredient lists to actual lists & join back to text for vectorizer

Combine both matrices

Apply to recipes

In [72]:
import pandas as pd

nodes_df = pd.read_csv('nodes_191120.csv')
print(nodes_df.head())
print(nodes_df.columns)

   node_id                    name  id   node_type  is_hub
0        0       1%_fat_buttermilk NaN  ingredient  no_hub
1        1   1%_fat_cottage_cheese NaN  ingredient  no_hub
2        3               10%_cream NaN  ingredient  no_hub
3        4               100%_bran NaN  ingredient  no_hub
4        5  10_inch_flour_tortilla NaN  ingredient  no_hub
Index(['node_id', 'name', 'id', 'node_type', 'is_hub'], dtype='object')


In [73]:
import pandas as pd

nodes_df = pd.read_csv('nodes_191120.csv')
id2name = dict(zip(nodes_df['node_id'].astype(str), nodes_df['name'].str.lower()))
name2id = dict(zip(nodes_df['name'].str.lower(), nodes_df['node_id'].astype(str)))

In [74]:
import re

def normalize_ingredient(ingredient):
    ingredient = ingredient.lower()
    ingredient = re.sub(r'\bfresh\b|\bdried\b|\bchopped\b|\bsliced\b|\bgrated\b|\bminced\b|\bpowdered\b|\bfrozen\b', '', ingredient)
    ingredient = ingredient.strip()
    if ingredient.endswith('s') and not ingredient.endswith('ss'):
        ingredient = ingredient[:-1]
    ingredient = ingredient.replace(' ', '_')
    return ingredient

In [75]:
def get_flavour_vector(ingredient_list, flavor_embeddings, name2id):
    embeds = []
    for ingredient in ingredient_list:
        normalized = normalize_ingredient(ingredient)
        node_id = name2id.get(normalized)
        if node_id and node_id in flavor_embeddings:
            embeds.append(flavor_embeddings[node_id])
    if not embeds:
        return [0.0] * len(next(iter(flavor_embeddings.values())))
    import numpy as np
    return np.mean(embeds, axis=0)

In [76]:
import ast

df_small['flavour_vector'] = df_small['ingredients'].apply(
    lambda x: get_flavour_vector(ast.literal_eval(x), flavor_embeddings, name2id)
)
df_small.to_csv('recipes_with_flavourgraph_vectors_first20.csv', index=False)

In [77]:
recipes_flavor = pd.read_csv('recipes_with_flavourgraph_vectors_first20.csv')

In [78]:
recipes_flavor

,id,name,ingredients,flavour_vector,flavor_profile
0,137739,arriba baked winter squash mexican style,"['winter squash', 'mexican seasoning', 'mixed ...",[-1.83971331e-01 -4.72420976e-02 8.09626095e-...,"{'spicy': 0.0, 'sweet': 0.26, 'umami': 0.1, 's..."
1,31490,a bit different breakfast pizza,"['prepared pizza crust', 'sausage patty', 'egg...",[-0.07182378 0.05897989 0.07096534 0.140941...,"{'spicy': 0.0, 'sweet': 0.3, 'umami': 0.1, 'so..."
2,112140,all in the kitchen chili,"['ground beef', 'yellow onions', 'diced tomato...",[-1.97033599e-01 1.24606217e-04 6.76930845e-...,"{'spicy': 0.13, 'sweet': 0.12, 'umami': 0.3, '..."
3,59389,alouette potatoes,"['spreadable cheese with garlic and herbs', 'n...",[-0.13329451 -0.06743051 0.01396198 -0.026733...,"{'spicy': 0.0, 'sweet': 0.04, 'umami': 0.06, '..."
4,44061,amish tomato ketchup for canning,"['tomato juice', 'apple cider vinegar', 'sugar...",[-4.84459065e-02 2.29501035e-02 4.57333028e-...,"{'spicy': 0.0, 'sweet': 0.43, 'umami': 0.0, 's..."
5,5289,apple a day milk shake,"['milk', 'vanilla ice cream', 'frozen apple ju...",[-1.10400394e-01 -8.55505839e-02 -5.64452447e-...,"{'spicy': 0.0, 'sweet': 0.55, 'umami': 0.05, '..."
6,25274,aww marinated olives,"['fennel seeds', 'green olives', 'ripe olives'...",[-1.72764480e-01 -5.87789863e-02 -9.00132284e-...,"{'spicy': 0.1, 'sweet': 0.25, 'umami': 0.3, 's..."
7,67888,backyard style barbecued ribs,"['pork spareribs', 'soy sauce', 'fresh garlic'...",[-1.46062031e-01 -3.75885330e-02 -6.23219740e-...,"{'spicy': 0.14, 'sweet': 0.39, 'umami': 0.24, ..."
8,70971,bananas 4 ice cream pie,"['chocolate sandwich style cookies', 'chocolat...",[ 1.15425192e-01 -5.70695708e-03 -1.00684687e-...,"{'spicy': 0, 'sweet': 0, 'umami': 0, 'sour': 0..."
9,75452,beat this banana bread,"['sugar', 'unsalted butter', 'bananas', 'eggs'...",[-7.78550953e-02 6.77323490e-02 -1.91745926e-...,"{'spicy': 0.0, 'sweet': 0.28, 'umami': 0.0, 's..."


In [79]:
flavor_labels = pd.read_csv('ingredient-flavor.csv')
print(flavor_labels.head())

   index      ingredient  spicy  sweet  umami  sour  salty
0      1            Salt   0.00    0.0    0.0   0.0    1.0
1      2           Sugar   0.00    1.0    0.0   0.0    0.0
2      3    Black Pepper   0.70    0.0    0.0   0.0    0.0
3      4  Cayenne Pepper   0.95    0.0    0.0   0.0    0.0
4      5    Chili Powder   0.80    0.0    0.0   0.0    0.0


In [80]:
ingredient_flavor_map = {}
for _, row in flavor_labels.iterrows():
    key = str(row['ingredient']).strip().lower() # safer for float/NaN
    ingredient_flavor_map[key] = {
        "spicy": row['spicy'],
        "sweet": row['sweet'],
        "umami": row['umami'],
        "sour": row['sour'],
        "salty": row['salty']
    }

In [81]:
def compose_flavor_profile(ingredient_list):
    profile = {"spicy": 0, "sweet": 0, "umami": 0, "sour": 0, "salty": 0}
    count = 0
    for ing in ingredient_list:
        ing_norm = ing.strip().lower()
        if ing_norm in ingredient_flavor_map:
            for key in profile:
                profile[key] += ingredient_flavor_map[ing_norm][key]
            count += 1
    if count > 0:
        for key in profile:
            profile[key] = round(profile[key]/count, 2)
    return profile

In [82]:
import ast

df_small['flavor_profile'] = df_small['ingredients'].apply(
    lambda x: compose_flavor_profile(ast.literal_eval(x))
)
print(df_small[['name', 'flavor_profile']].head(10))

                                         name  \
0  arriba   baked winter squash mexican style   
1            a bit different  breakfast pizza   
2                   all in the kitchen  chili   
3                          alouette  potatoes   
4          amish  tomato ketchup  for canning   
5                     apple a day  milk shake   
6                       aww  marinated olives   
7              backyard style  barbecued ribs   
8                    bananas 4 ice cream  pie   
9                     beat this  banana bread   

                                      flavor_profile  
0  {'spicy': 0.0, 'sweet': 0.26, 'umami': 0.1, 's...  
1  {'spicy': 0.0, 'sweet': 0.3, 'umami': 0.1, 'so...  
2  {'spicy': 0.13, 'sweet': 0.12, 'umami': 0.3, '...  
3  {'spicy': 0.0, 'sweet': 0.04, 'umami': 0.06, '...  
4  {'spicy': 0.0, 'sweet': 0.43, 'umami': 0.0, 's...  
5  {'spicy': 0.0, 'sweet': 0.55, 'umami': 0.05, '...  
6  {'spicy': 0.1, 'sweet': 0.25, 'umami': 0.3, 's...  
7  {'spicy': 0.14, '

In [83]:
df_small.to_csv('recipes_with_flavor_profiles.csv', index=False)